In [1]:
import os
import shutil

import xlwings as xw
import pandas as pd

# 作成ライブラリの読み込み
from lib.lib_directory import init_dir
from lib.lib_shape import get_shape_dict, add_shape_info, split_shape_line_dict, add_shape_info_point, add_shape_info_lineno
from lib.lib_shape_process import process_shape, process_shape_process, process_shape_judgement, process_shape_loop, search_shape

### 初期設定(実行前に確認・変更を行ってください)

In [2]:
#########################################
# 初期設定(実行前に確認・変更を行ってください) #
#########################################

# データフローを記載するExcelファイル名
filename = './sample.xlsx'

# workflowのディレクトリ構造を出力ディレクトリ
output_dir = './output/'

# 各種設定のtemplateを格納しているディレクトリ
template_dir = './template/'

# workflowのプロジェクト名
project_name = 'sample_project02'

### 以降、プログラム処理の部分のため必要がない場合は修正を行わない

In [3]:
##############################################
# 初期設定(固定値のため基本的に変更しないでください) #
##############################################

# 図形の付加情報
dict_fix_name = {
    'Rectangle':{'desc':'処理', 'kubun':0}
    , 'Diamond':{'desc':'判定', 'kubun':1}
    , 'Flowchart: Decision':{'desc':'判定終端', 'kubun':-1}
    , 'Trapezoid':{'desc':'繰り返し', 'kubun':2}
    , 'Flowchart: Manual Operation':{'desc':'繰り返し終端', 'kubun':-2}
    , 'Straight Connector':{'desc':'直線', 'kubun':99}
    , 'Elbow Connector':{'desc':'カギ線', 'kubun':99}
}

# ネスト時の文字列
nest_str = '    '

In [4]:
# workflowディレクトリの生成
base_dir = output_dir + project_name
init_dir(base_dir, output_dir, project_name)

# mainとなるdigのファイルをオープン
fp = open(base_dir + '/' + project_name + '.dig', 'w')

# template_management.csvを取り込み
df = pd.read_csv(template_dir + 'template_management.csv')

# Excelデータの読み込み(フロー記述のExcel)
wb = xw.Book(filename)
sheet = wb.sheets.active

# Excelデータの読み込み(templateのExcel)


In [5]:
# 図形情報の辞書化
dict_org = {}
dict_org = get_shape_dict(sheet)

# 図形付加情報,X軸下端,Y軸右端を追加
dict_org = add_shape_info(dict_org, dict_fix_name)

# 図形と線を分離
dict_shape = {}
dict_line = {}
dict_shape, dict_line = split_shape_line_dict(dict_org)

# 図形のin_point/out_pointを付与
dict_shape = add_shape_info_point(dict_shape)

# 図形のin_line_no/out_line_noを付与
dict_shape = add_shape_info_lineno(dict_shape, dict_line)

In [6]:
'''
# 図形の処理：処理(四角)
def process_shape_process(current_shape, nest):

    #　タスク名を取得
    task_name = dict_shape[current_shape]['text'][0]
    
    # 処理タイプを取得
    type_name = dict_shape[current_shape]['text'][1]

    # 管理ファイルから情報を取得
    management_info = df[df['type'] == type_name]

    # テンプレートファイルをコピー(exe)
    print(management_info['exe_filename'].iloc[0].split('.'))
    extension = management_info['exe_filename'].iloc[0].split('.')[1]
    src = template_dir + management_info['exe_filename'].iloc[0]
    dest = '{}/{}/{}_{}.{}'.format(base_dir, management_info['exe_dir'].iloc[0], task_name, type_name, extension)
    shutil.copy(src, dest)

    # テンプレートファイルをコピー(dig)
    src = template_dir + management_info['dig_filename'].iloc[0]
    dig_filename = '{}_{}.dig'.format(task_name, type_name)
    dest = '{}/digs/{}'.format(base_dir, dig_filename)
    shutil.copy(src, dest)

    # メインとなるdigファイルに出力
    if len(dict_shape[current_shape]['text']) > 2:
        fp.write('{}# {}\n'.format(nest_str * nest, dict_shape[current_shape]['text'][2]))
    fp.write('{}+{}:\n'.format(nest_str * nest, task_name))
    fp.write('{}  call>: digs/{}\n\n'.format(nest_str * nest, dig_filename))
'''

"\n# 図形の処理：処理(四角)\ndef process_shape_process(current_shape, nest):\n\n    #\u3000タスク名を取得\n    task_name = dict_shape[current_shape]['text'][0]\n    \n    # 処理タイプを取得\n    type_name = dict_shape[current_shape]['text'][1]\n\n    # 管理ファイルから情報を取得\n    management_info = df[df['type'] == type_name]\n\n    # テンプレートファイルをコピー(exe)\n    print(management_info['exe_filename'].iloc[0].split('.'))\n    extension = management_info['exe_filename'].iloc[0].split('.')[1]\n    src = template_dir + management_info['exe_filename'].iloc[0]\n    dest = '{}/{}/{}_{}.{}'.format(base_dir, management_info['exe_dir'].iloc[0], task_name, type_name, extension)\n    shutil.copy(src, dest)\n\n    # テンプレートファイルをコピー(dig)\n    src = template_dir + management_info['dig_filename'].iloc[0]\n    dig_filename = '{}_{}.dig'.format(task_name, type_name)\n    dest = '{}/digs/{}'.format(base_dir, dig_filename)\n    shutil.copy(src, dest)\n\n    # メインとなるdigファイルに出力\n    if len(dict_shape[current_shape]['text']) > 2:\n        fp.write

In [7]:
'''
# 図形の処理：判定(ひし形)
# flag = 0：if処理 flag = 1：else処理
def process_shape_judgement(current_shape, nest, flag):

    if flag == 0:
        #　タスク名を取得
        task_name = dict_shape[current_shape]['text'][0]

        # メインとなるdigファイルに出力
        if len(dict_shape[current_shape]['text']) > 1:
            fp.write('{}# {}\n'.format(nest_str * nest, dict_shape[current_shape]['text'][1]))

        fp.write('{}+{}:\n'.format(nest_str * nest, task_name))
        fp.write('{}  if>: \n'.format(nest_str * nest))
        fp.write('{}  _do:\n\n'.format(nest_str * nest))
    
    else:
        # メインとなるdigファイルに出力
        fp.write('{}  _else_do:\n\n'.format(nest_str * nest))
'''

"\n# 図形の処理：判定(ひし形)\n# flag = 0：if処理 flag = 1：else処理\ndef process_shape_judgement(current_shape, nest, flag):\n\n    if flag == 0:\n        #\u3000タスク名を取得\n        task_name = dict_shape[current_shape]['text'][0]\n\n        # メインとなるdigファイルに出力\n        if len(dict_shape[current_shape]['text']) > 1:\n            fp.write('{}# {}\n'.format(nest_str * nest, dict_shape[current_shape]['text'][1]))\n\n        fp.write('{}+{}:\n'.format(nest_str * nest, task_name))\n        fp.write('{}  if>: \n'.format(nest_str * nest))\n        fp.write('{}  _do:\n\n'.format(nest_str * nest))\n    \n    else:\n        # メインとなるdigファイルに出力\n        fp.write('{}  _else_do:\n\n'.format(nest_str * nest))\n"

In [8]:
'''
# 図形の処理：繰り返し(台形)
def process_shape_loop(current_shape, nest):
    
    #　タスク名を取得
    task_name = dict_shape[current_shape]['text'][0]
    
    # 処理タイプを取得
    type_name = dict_shape[current_shape]['text'][1]

    # 管理ファイルから情報を取得
    management_info = df[df['type'] == type_name]

    # テンプレートファイルをコピー(exe)
    if len(management_info['exe_filename'].iloc[0]) != 0:
        extension = management_info['exe_filename'].iloc[0].split('.')[1]
        src = template_dir + management_info['exe_filename'].iloc[0]
        dest = '{}/{}/{}_{}.{}'.format(base_dir, management_info['exe_dir'].iloc[0], task_name, type_name, extension)
        shutil.copy(src, dest)
    
    # メインとなるdigファイルに出力
    if len(dict_shape[current_shape]['text']) > 2:
        fp.write('{}# {}\n'.format(nest_str * nest, dict_shape[current_shape]['text'][2]))

    fp.write('{}+{}:\n'.format(nest_str * nest, task_name))
    management_info = df[df['type'] == type_name]
    with open(template_dir + management_info['dig_filename'].iloc[0], 'r') as f:
        for line in f:
            if '{{file_name}}' in line:
                output = line.replace('{{file_name}}', '{}/{}_{}.{}'.format(management_info['exe_dir'].iloc[0], task_name, type_name, extension))
            else:
                output = line
            fp.write('{}{}\n'.format(nest_str * nest, output))
    fp.write('\n')
'''

"\n# 図形の処理：繰り返し(台形)\ndef process_shape_loop(current_shape, nest):\n    \n    #\u3000タスク名を取得\n    task_name = dict_shape[current_shape]['text'][0]\n    \n    # 処理タイプを取得\n    type_name = dict_shape[current_shape]['text'][1]\n\n    # 管理ファイルから情報を取得\n    management_info = df[df['type'] == type_name]\n\n    # テンプレートファイルをコピー(exe)\n    if len(management_info['exe_filename'].iloc[0]) != 0:\n        extension = management_info['exe_filename'].iloc[0].split('.')[1]\n        src = template_dir + management_info['exe_filename'].iloc[0]\n        dest = '{}/{}/{}_{}.{}'.format(base_dir, management_info['exe_dir'].iloc[0], task_name, type_name, extension)\n        shutil.copy(src, dest)\n    \n    # メインとなるdigファイルに出力\n    if len(dict_shape[current_shape]['text']) > 2:\n        fp.write('{}# {}\n'.format(nest_str * nest, dict_shape[current_shape]['text'][2]))\n\n    fp.write('{}+{}:\n'.format(nest_str * nest, task_name))\n    management_info = df[df['type'] == type_name]\n    with open(template_dir + ma

In [9]:
'''
# 図形の処理
def process_shape(current_shape, nest, ):
    # 処理：四角
    if dict_shape[current_shape]['add_info']['kubun'] == 0:
        process_shape_process(current_shape, nest, dict_shape, template_dir, base_dir, fp, df, nest_str)
        
    # 判定：ひし形
    elif dict_shape[current_shape]['add_info']['kubun'] == 1:
        process_shape_judgement(current_shape, nest, 0, dict_shape, fp, nest_str)
    
    # 繰り返し：台形
    elif dict_shape[current_shape]['add_info']['kubun'] == 2:
        process_shape_loop(current_shape, nest, dict_shape, df, template_dir, base_dir, nest_str, fp)
'''

"\n# 図形の処理\ndef process_shape(current_shape, nest, ):\n    # 処理：四角\n    if dict_shape[current_shape]['add_info']['kubun'] == 0:\n        process_shape_process(current_shape, nest, dict_shape, template_dir, base_dir, fp, df, nest_str)\n        \n    # 判定：ひし形\n    elif dict_shape[current_shape]['add_info']['kubun'] == 1:\n        process_shape_judgement(current_shape, nest, 0, dict_shape, fp, nest_str)\n    \n    # 繰り返し：台形\n    elif dict_shape[current_shape]['add_info']['kubun'] == 2:\n        process_shape_loop(current_shape, nest, dict_shape, df, template_dir, base_dir, nest_str, fp)\n"

In [10]:
'''
# 図形の探索
def search_shape(list_fin, list_stack_process, list_stack_termination, nest):
    #print('==========')
    #print('list_fin：' + str(list_fin))
    #print('list_stack_process：' + str(list_stack_process))
    #print('list_stack_termination：' + str(list_stack_termination))
    #print('nest：' + str(nest))
        
    # スタックされている処理を取得
    if len(list_stack_process) != 0:
        current_shape = list_stack_process.pop()
        process_shape(current_shape, nest)
        
        # 判定か繰り返しを処理した場合、nestをインクリメント
        if dict_shape[current_shape]['add_info']['kubun'] in [1, 2]:
            nest += 1
        list_fin.append(current_shape)
            
    # 終端図形の番号を取得し、nestをデクリメント
    elif len(list_stack_termination) != 0:
        while len(list_stack_termination) != 0:
            current_shape = list_stack_termination.pop()
            if current_shape not in list_fin:
                list_fin.append(current_shape)
                nest -= 1
                while 1:
                    try:
                        list_stack_termination.remove(current_shape)
                    except ValueError as e:
                        break
                break

    # 全て処理し終えていたらreturn
    if len(list_fin) == dict_shape['cnt']:
        return list_fin, list_stack_process, list_stack_termination, nest
    
    # out_line_noから次の図形を探索
    for out_line in dict_shape[current_shape]['out_line_no']:
        i = 0
                
        for i in range (0, dict_shape['cnt']):
            if out_line in dict_shape[i]['in_line_no']:
                # 見つかった図形が終端の場合は終端リストに格納
                if dict_shape[i]['add_info']['kubun'] < 0:
                    list_stack_termination.append(i)
                
                # 見つかった図形が終端ではない場合再帰処理
                else:
                    # 判定の2つめのout_line_noが終端以外に紐付いている場合、else句
                    if dict_shape[current_shape]['add_info']['kubun'] == 1 and dict_shape[current_shape]['out_line_no'][1] == out_line:
                        process_shape_judgement(current_shape, nest - 1, 1, dict_shape, fp, nest_str)
                    list_stack_process.append(i)
                    list_fin, list_stack_process, list_stack_termination, nest = search_shape(list_fin, list_stack_process, list_stack_termination, nest)
                break

    return list_fin, list_stack_process, list_stack_termination, nest
'''

"\n# 図形の探索\ndef search_shape(list_fin, list_stack_process, list_stack_termination, nest):\n    #print('==========')\n    #print('list_fin：' + str(list_fin))\n    #print('list_stack_process：' + str(list_stack_process))\n    #print('list_stack_termination：' + str(list_stack_termination))\n    #print('nest：' + str(nest))\n        \n    # スタックされている処理を取得\n    if len(list_stack_process) != 0:\n        current_shape = list_stack_process.pop()\n        process_shape(current_shape, nest)\n        \n        # 判定か繰り返しを処理した場合、nestをインクリメント\n        if dict_shape[current_shape]['add_info']['kubun'] in [1, 2]:\n            nest += 1\n        list_fin.append(current_shape)\n            \n    # 終端図形の番号を取得し、nestをデクリメント\n    elif len(list_stack_termination) != 0:\n        while len(list_stack_termination) != 0:\n            current_shape = list_stack_termination.pop()\n            if current_shape not in list_fin:\n                list_fin.append(current_shape)\n                nest -= 1\n               

In [11]:
list_fin = []
list_stack_process = []
list_stack_termination = []
nest = 0

# 図形の処理を全て終わるまでループ
while len(list_fin) != dict_shape['cnt']:
    
    # はじめの図形の処理
    if len(list_fin) == 0 and len(list_stack_process) == 0 and len(list_stack_termination) == 0:
        i = 0
        for i in range (0, dict_shape['cnt']):
            if len(dict_shape[i]['in_line_no']) == 0:
                list_stack_process.append(i)
                break
        continue
    
    # 図形の探索
    list_fin, list_stack_process, list_stack_termination, nest = search_shape(list_fin, list_stack_process, list_stack_termination, nest, dict_shape, fp, nest_str, template_dir, base_dir, df)

['template_exe_query', 'sql']


TypeError: search_shape() missing 5 required positional arguments: 'fp', 'nest_str', 'template_dir', 'base_dir', and 'df'

In [17]:
fp.close()